In [1]:
# CrossCompute
target_folder = '/tmp'

# Prompts users to enter a zipcode in the tool
# The default zipcode is 11419

Please_enter_a_zipcode = 11419

Color_scheme_select = """
    streets
    
    streets
    light
    dark
    satellite
    streets-satellite
    wheatpaste
    streets-basic
    comic
    outdoors
    run-bike-hike
    pencil
    pirates
    emerald
    high-contrast
"""

In [2]:
if len(Color_scheme_select.strip().splitlines()) >= 1:
    look = Color_scheme_select.strip().splitlines()[0]
else :
    look = "streets"

In [3]:
import subprocess
import sys

# This function is used to install packages using pip
# It's equivalent to doing 'pip install ______'
def install(package):
    subprocess.call([sys.executable, "-m", "pip", "install", package])

install('sodapy') # Package for NYC OpenData API


In [4]:
import pandas as pd
from sodapy import Socrata # Used to access/ work with NYCOpenData API

In [5]:
#################################
# WORKING WITH CATCH BASIN DATA #
#################################


# Grabbing data from API
client = Socrata("data.cityofnewyork.us",
                'YFHnlAd1f74IprxACGOlr46td',
                username="nycopendataninjas@gmail.com",
                password="DataNinjas4TheWin!")

# Limits the data to only clogged catch basin complaints in a specified zipcode^
results = client.get("fhrw-4uyv", 
                     incident_zip = Please_enter_a_zipcode,
                     complaint_type="Sewer",
                     descriptor = "Catch Basin Clogged/Flooding (Use Comments) (SC)",
                     limit=10000)

# Convert to pandas DataFrame
df_threeOneOneReq = pd.DataFrame.from_records(results)

# Only gets the location of these complaints
complaintLoc = df_threeOneOneReq[['latitude','longitude']]

In [10]:
#################################
# WORKING WITH TREE CENSUS DATA #
#################################


# Limits the data to only trees that are ALIVE in that specified zipcode that was entered above^
results = client.get("5rq2-4hqu",
                     zipcode = Please_enter_a_zipcode,
                     status = 'Alive',
                     limit=10000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# Only get the columns that are useful
results_df = results_df[['tree_dbh', 'health','status','latitude','longitude','spc_latin','boro_ct']]            
            
# Replaces words with numbers so that it is easier to create a 'grade' for each tree
results_df = results_df.replace(['Poor','Fair','Good'],[0,50,100])

# 'tree_dbh' was an object, this converts it to an int so that it can be added to 'health' and 'status'
results_df['tree_dbh'] = pd.to_numeric(results_df['tree_dbh'])

# Anywhere there is an 'NaN', make it a zero
results_df = results_df.fillna(0)

# Looks through list of each species and it's type
df = pd.read_csv('Species_Types.csv')
df = df.set_index('Species')

# Decides whether each tree is deciduous, conferous, etc.
results_df['Type'] = df.loc[results_df.spc_latin,'Type'].values

# Replaces words with numbers so that it is easier to create a 'grade' for each tree
results_df = results_df.replace(['deciduous','coniferous','evergreen','both'],[1,0,0,0])

# Generates a final grade that will be the value of the weight on the heat map for each tree
results_df['Final Grade'] = ((results_df.tree_dbh + results_df.health)/100)*results_df.Type

# Removes all the trees that dont lose leaves
results_df = results_df[results_df.Type != 0]
results_df = results_df.fillna(0)
results_df[['tree_dbh','health','health','spc_latin','Final Grade']]

,tree_dbh,health,health,spc_latin,Final Grade
0,13,100,100,Metasequoia glyptostroboides,1.13
1,4,50,50,Metasequoia glyptostroboides,0.54
2,3,100,100,Ulmus americana,1.03
3,15,100,100,Gleditsia triacanthos var. inermis,1.15
4,18,50,50,Gleditsia triacanthos var. inermis,0.68
5,12,100,100,Gleditsia triacanthos var. inermis,1.12
6,24,100,100,Quercus palustris,1.24
7,21,50,50,Quercus palustris,0.71
8,4,100,100,Quercus palustris,1.04
9,26,100,100,Quercus palustris,1.26


In [30]:
list= results_df['boro_ct'].unique()
df2 = pd.DataFrame(list)
g1 = results_df.groupby('boro_ct', as_index=False)['Final Grade'].mean()
g1= g1.set_index('boro_ct')

In [31]:
# Gets all of the unique blocks from that zipcode
listOfAllBlocks = results_df.boro_ct.unique()


#Saves and outputs a map
from geotable import GeoTable
t = GeoTable.load("Shape_Files/geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c.shp")
t['bctcb2010'] = t['bctcb2010'].str[:7]
t= t.loc[t['bctcb2010'].isin(listOfAllBlocks)]
t = t.reset_index()

In [32]:
t['localGrade'] = g1.loc[t.bctcb2010,'Final Grade'].values
t['FillRedsFromMean'] = t['localGrade']
t

,index,bctcb2010,boro_code,boro_name,cb2010,ct2010,shape_area,shape_leng,geometry_object,geometry_layer,geometry_proj4,localGrade,FillRedsFromMean
0,14888,4009600,4,Queens,1000,009600,217328.639367,2192.690439,"POLYGON ((-73.8310343018354 40.68217205000504,...",geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,1.029706,1.029706
1,14890,4009800,4,Queens,1000,009800,141091.881535,1642.906480,POLYGON ((-73.82593749776326 40.68593649651108...,geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,0.970282,0.970282
2,14892,4010000,4,Queens,1000,010000,173540.122964,1869.578757,POLYGON ((-73.82633041700251 40.67908393951462...,geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,1.071882,1.071882
3,14895,4010400,4,Queens,1000,010400,201342.307765,2133.476165,POLYGON ((-73.82495350795382 40.68390244993241...,geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,1.135859,1.135859
4,14896,4010600,4,Queens,1000,010600,198395.095443,2066.351023,POLYGON ((-73.82501128938837 40.68795231792618...,geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,1.060156,1.060156
5,14897,4010800,4,Queens,1000,010800,231491.871520,2444.663134,POLYGON ((-73.82933645724084 40.68900399665638...,geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,0.980738,0.980738
6,14898,4011000,4,Queens,1000,011000,296371.946400,2753.453526,"POLYGON ((-73.8349901302407 40.69027950101693,...",geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,1.042889,1.042889
7,14906,4011800,4,Queens,1000,011800,160759.499905,1779.418584,POLYGON ((-73.83052645683836 40.69143047239994...,geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,0.896494,0.896494
8,14930,4014800,4,Queens,1000,014800,94898.702843,1261.254649,POLYGON ((-73.81570906178112 40.69563808357589...,geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,1.061677,1.061677
9,14932,4015000,4,Queens,1000,015000,116511.949997,1432.433897,POLYGON ((-73.81512084958932 40.69443964348664...,geo_export_0c48d94e-1efc-4997-a51f-34df0cb1a82c,+proj=longlat +ellps=WGS84 +no_defs,0.960131,0.960131


In [33]:
target_path = t.save_csv(target_folder + '/choropleth.csv')
print('NYC_streets_'+look+'_geotable_path = %s' % target_path)

NYC_streets_streets_geotable_path = /tmp/choropleth.csv


In [1]:
link = ("https://sites.google.com/view/leaflessnyc/zipcodes/"+str(Please_enter_a_zipcode))
linkList = link.split()

linkDF = pd.DataFrame(linkList, columns=['Please visit the link below for a heatmap'])
linkDF.set_index('Please visit the link below for a heatmap', inplace=True)

target_path1 = linkDF.to_csv(target_folder + '/linkDF.csv')
from os.path import join
target_path = join(target_folder, 'linkDF.csv')
print('HeatMap_Link_table_path = ' + target_path)

NameError: name 'Please_enter_a_zipcode' is not defined